# Übungsblatt 4


In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import math 
from objdict import ObjDict

%load_ext autoreload
%autoreload

## Aufgabe 3.1

Zeigen Sie, dass sich dasselbe Ergebnis auch mit einer Assemblierungsmatrix erzielen
lasst.

K = zeros(6, 6);
I = [6, 1];
Ke = [1, 2; 3, 4];
K(I, I) = K(I, I) + Ke;

In [2]:
# Übersetzt in Python
K = np.zeros((6,6))
I = [5,0]
Ke = np.array([[1,2],[3,4]])
K[np.ix_(I,I)] += Ke 
#print(K)

def addKe(K, index, Ke):
    for i, I in enumerate(index):
        for j, J in enumerate(index):
            K[I, J] += Ke[i, j]
    return K

Ne = 5
K = np.zeros((Ne, Ne))
I = [Ne-1, 0] 
Ke = np.array([[1, 2], [3, 4]]) #lokale Steifigkeitsmatrix Ke

K = addKe(K, I, Ke) #lokale Matrix Ke auf Globale Matrix überführen
print(K)

[[4. 0. 0. 0. 3.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [2. 0. 0. 0. 1.]]


## Aufgabe 3.2

Erzeugen Sie in Matlab die FE-Datenstruktur für das Netz aus Aufgabe 1.1 (es sei
x3 = 0 und x2 = 20). Geben Sie mithilfe einer Schleife die Knotenkoordinaten aller
Elemente aus.

In [3]:
s = ObjDict()
s.nodes = np.array([5, 20, 0, 10, 15])
s.elements = np.array([[1,5,4,3],[4,2,5,1]])

Ne = len(s.elements[0])+1

cords = []
for i in np.array(range(1, Ne)): 
    I = np.array(s.elements[:,i-1])
    I = I.reshape([2,1])
    xi = s.nodes[I.flatten()-1]
    cords.append(xi)

print(cords)

[array([ 5, 10]), array([15, 20]), array([10, 15]), array([0, 5])]


## Aufgabe 3.3
Implementieren Sie die Funktionen pileKe, pileRe und assembleKr wie im Video
erläutert. Nutzen Sie ein Live-Skript um die Funktionen zu testen.

In [4]:
s = ObjDict()
s.E = 35e9  
s.d = 0.8  
s.l = 20  
s.F = 2e6  
s.rho = 2500  
s.g = 9.81  
s.C = 1.75e7  
s.S = 120e6
s.A = np.pi * (s.d / 2)**2
s.EA = s.E * s.A  
s.n = s.g * s.rho * s.A
s.nodes = np.array([[0,10,20],[0,0,0]])
s.elements = np.array([[0,1],[1,2]])

from pile import *

xe = s.nodes[:,s.elements[:,1]]

s.keFunc = pileKe(s)
s.keFunc(xe)

s.reFunc = pileRe(s)
s.reFunc(xe)

from fem import assembleKr

K, r = assembleKr(s)
print(K)
print(r)



[[ 1.81762522e+09 -1.73012522e+09  0.00000000e+00]
 [-1.73012522e+09  3.63525044e+09 -1.73012522e+09]
 [ 0.00000000e+00 -1.73012522e+09  1.81762522e+09]]
[ 61638.04786343 123276.09572686  61638.04786343]


## Aufgabe 3.4
Kopieren Sie den Programmcode von Übungsblatt 3 und ändern Sie die Funktion
pileFEM dahingehend, dass die Berechnung mithilfe der Funktionen aus Aufgabe 3.3
erfolgt.

In [11]:
from fem import *
from pile import *

s = ObjDict()
s.E = 35e9  
s.d = 0.8  
s.l = 20  
s.F = 2e6  
s.rho = 2500  
s.g = 9.81  
s.C = 1.75e7  
s.S = 120e6
s.A = np.pi * (s.d / 2)**2
s.EA = s.E * s.A  
s.n = s.g * s.rho * s.A


k = 2

def pileFEM2(s,k):
    m = ObjDict()
    m.nodes = np.array([np.linspace(0,s.l,k+1),np.linspace(0,0,k+1)])
    m.elements = np.array([np.arange(0,k,1,),np.arange(1,k+1,1)])
    m.keFunc = pileKe(s)
    m.reFunc = pileRe(s)

    K, r = assembleKr(m)

    print(K)
    print(r)

    K[-1, -1] += s.S
    r[0] += s.F

    print(K)
    print(r)

    uHatK = np.linalg.solve(K,r)
    xK = np.linspace (0, s.l, k+1)
    return xK, uHatK

#pileFEM2(s,k)

xK,uHatK = pileFEM2(s,k)

#print(xK)
#print(uHatK)

[[ 1.81762522e+09 -1.73012522e+09  0.00000000e+00]
 [-1.73012522e+09  3.63525044e+09 -1.73012522e+09]
 [ 0.00000000e+00 -1.73012522e+09  1.81762522e+09]]
[ 61638.04786343 123276.09572686  61638.04786343]
[[ 1.81762522e+09 -1.73012522e+09  0.00000000e+00]
 [-1.73012522e+09  3.63525044e+09 -1.73012522e+09]
 [ 0.00000000e+00 -1.73012522e+09  1.93762522e+09]]
[2061638.04786343  123276.09572686   61638.04786343]
